## HWW5:

В домашньому завданні до даного модулю ви потренуєтесь робити тестове завдання для влаштування на роботу. 
За даними акселерометра з мобільного телефону потрібно класифікувати, якою діяльністю займається людина: йде, стоїть, біжить чи йде по сходах. 
Знайти датасет ви можете за посиланням....

Використайте алгоритми SVM та випадковий ліс з бібліотеки scikit-learn. Як характеристики можете брати показники з акселерометра, проте щоб покращити результати роботи алгоритмів, спочатку можна підготувати наш датасет і розрахувати часові ознаки (time domain features). Більше ці характеристики описані в даній статті.

Порівняйте результати роботи обох алгоритмів на різних фічах та різні моделі між собою.

In [106]:
import math as m
from pathlib import Path
from typing import Union

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
%matplotlib inline

In [108]:
def read_from_csvfile(file: Path) -> pd.DataFrame:
    """Read content from csv-file and return dataframe from content."""
    df = pd.read_csv(file)
    
    return df

max_x, min_x, entropy_x, Interquartile_Range_x, max_y, min_y, Mean_of_absolute_deviation_y, Median_y, Skewness_y, Standard_deviation_y, Root_mean_square_error_y, Skewness_z

In [109]:
df = pd.DataFrame()
classification_human_activity = {}

- https://www.geeksforgeeks.org/different-ways-to-create-pandas-dataframe/
- https://pandas.pydata.org/docs/reference/api/pandas.concat.html

In [116]:
def get_time_domain_features(y: str, df: pd.DataFrame) -> pd.DataFrame:
    if y not in classification_human_activity:
        classification_human_activity[y] = len(classification_human_activity)

    data = {
            'activity': [classification_human_activity[y]],
            'max_x': [max(df['accelerometer_X'])],
            'min_x': [df['accelerometer_X'].describe().loc['min']],
            'entropy_x': [-m.log(1/df['accelerometer_X'].shape[0])],
            'Interquartile_Range_x': [df['accelerometer_X'].describe().loc['25%']],
            'max_y': [df['accelerometer_Y'].describe().loc['max']],
            'min_y': [df['accelerometer_Y'].describe().loc['min']],
            'Mean_of_absolute_deviation_y': [df['accelerometer_Y'].describe().loc['mean']],
            'Median_y': [df['accelerometer_Y'].describe().loc['50%']],
            'Skewness_y': [1/df['accelerometer_Y'].describe().loc['std']],
            'Standard_deviation_y': [df['accelerometer_Y'].describe().loc['std']],
            'Root_mean_square_error_y': [abs(df['accelerometer_Y'].describe().loc['mean'])**0.5],
            'Skewness_z': [1/df['accelerometer_Z'].describe().loc['std']],
            }
    df_f = pd.DataFrame(data)
    
    return df_f

In [117]:
def get_data_to_dafaframe(path: Union[str, Path], df: pd.DataFrame) -> pd.DataFrame:
    path = Path(path)
    if not path.is_dir() or not path.exists():
        return None
    
    for file_system_object in path.iterdir():
        if file_system_object.is_dir():
            df = get_data_to_dafaframe(file_system_object, df)
            
        elif file_system_object.suffix.lower() in ('.csv',):
            df_add = read_from_csvfile(file_system_object)
            df = pd.concat([df, get_time_domain_features(file_system_object.stem.split('-')[0], df_add)], ignore_index=True)

    return df

In [118]:
df = get_data_to_dafaframe('data', df)

In [120]:
classification_human_activity

{'walking': 0, 'running': 1, 'idle': 2, 'stairs': 3}

In [119]:
df

,activity,max_x,min_x,entropy_x,Interquartile_Range_x,max_y,min_y,Mean_of_absolute_deviation_y,Median_y,Skewness_y,Standard_deviation_y,Root_mean_square_error_y,Skewness_z
0,0,4.141969,-15.088259,3.401197,-6.275202,-1.470040,-17.980455,-9.659965,-10.249578,0.244852,4.084106,3.108048,0.146323
1,0,8.475474,-11.669339,3.401197,-3.555389,-0.311246,-22.443247,-10.444944,-9.866505,0.162977,6.135832,3.231864,0.103948
2,0,14.877569,-6.143522,3.401197,-3.241749,0.311246,-27.140670,-11.915463,-12.035652,0.152704,6.548619,3.451878,0.101890
3,0,2.245761,-15.026010,3.401197,-8.042123,8.834604,-28.773516,-10.401370,-11.935095,0.130572,7.658616,3.225115,0.109849
4,0,1.302446,-11.808203,3.401197,-5.091270,3.586514,-16.314090,-8.448340,-8.980650,0.232550,4.300150,2.906603,0.177061
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6457,3,8.398860,-6.224924,3.401197,-0.689530,-2.715025,-17.501614,-8.694463,-9.035717,0.311788,3.207309,2.948638,0.182492
6458,3,12.196063,-6.354211,3.401197,1.230620,3.016694,-21.049822,-9.039388,-8.477868,0.190911,5.238048,3.006558,0.315002
6459,3,8.073248,-4.582502,3.401197,1.031901,2.973599,-17.300500,-7.161376,-7.153874,0.212308,4.710145,2.676075,0.221586
6460,3,9.035717,-6.904877,3.401197,-1.915361,-1.489193,-22.720974,-9.817344,-9.631874,0.189880,5.266475,3.133264,0.224073
